In [1]:
import tensorflow as tf
import pandas as pd
import keras
train = pd.read_csv("dataset_train_set.csv")
test = pd.read_csv("dataset_dev_set.csv")

Using TensorFlow backend.


In [2]:
X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [3]:
X_train = X_train[['aPosLS','hPosLS','hGD5','aSpent','aPtsPG','hSPG','hGCPG','aPos','aSTPG','aGSPG','hSTPG','aSPG','aGD7','hPos','aGD5','hPtsPG','hSpent','aPTSPG7','hGD7']].copy()
X_test = X_test[['aPosLS','hPosLS','hGD5','aSpent','aPtsPG','hSPG','hGCPG','aPos','aSTPG','aGSPG','hSTPG','aSPG','aGD7','hPos','aGD5','hPtsPG','hSpent','aPTSPG7','hGD7']].copy()

In [4]:
X_train.head()

,aPosLS,hPosLS,hGD5,aSpent,aPtsPG,hSPG,hGCPG,aPos,aSTPG,aGSPG,hSTPG,aSPG,aGD7,hPos,aGD5,hPtsPG,hSpent,aPTSPG7,hGD7
0,18,18,0.0,18.07,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,18.72,0.0,0.0
1,9,5,0.0,8.92,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,47.15,0.0,0.0
2,12,14,0.0,29.52,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,17.10,0.0,0.0
3,15,16,0.0,0.00,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,11.99,0.0,0.0
4,13,3,0.0,31.03,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,47.84,0.0,0.0


In [5]:
print(X_train.shape)
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[1000]

(4180, 19)


array([ 0.20521545,  0.24625854, -0.00820862,  0.16622452,  0.05294559,
        0.51131482,  0.03940137,  0.49251709,  0.21720004,  0.05294559,
        0.2685039 ,  0.4446198 ,  0.04104309,  0.16417236,  0.03283447,
        0.07018369,  0.        ,  0.08208618,  0.00586916])

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=3, min_samples_leaf=5)
clf_gini.fit(X_train, y_train)
y_pred = clf_gini.predict(X_test)

print("Accuracy is {}".format(accuracy_score(y_test,y_pred)*100))

Accuracy is 51.90789473684211


In [7]:
y_train.value_counts()

2    1971
0    1120
1    1089
Name: target, dtype: int64

In [8]:
whome = 1
wdraw = 1971/1089
waway = 1971/1120
print("{},{},{}".format(whome,wdraw,waway))

1,1.8099173553719008,1.7598214285714286


In [9]:
class_weights = {0:1.7598,
                 1:1.8099,
                 2:1}

In [10]:
total = 0
for i in range(10):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(3, activation = tf.nn.softmax))
    model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
    
    model.fit(X_train_norm, y_train.values, epochs=50, validation_data=(X_test_norm,y_test), class_weight=class_weights, verbose=0)
    
    val_loss, val_acc = model.evaluate(X_test_norm, y_test)
    print(val_loss, val_acc)
    total += val_acc
print("Final accuracy: {}".format(total/10))

1520/1520 [==============================] - 0s 17us/step
0.9972658696927522 0.5125
1520/1520 [==============================] - 0s 16us/step
0.9970496064738223 0.5144736842105263
1520/1520 [==============================] - 0s 45us/step
0.9929651636826364 0.5256578947368421
1520/1520 [==============================] - 0s 31us/step
0.9936537504196167 0.5197368421052632
1520/1520 [==============================] - 0s 18us/step
1.0042003468463296 0.5059210526315789
1520/1520 [==============================] - 0s 23us/step
0.991322710639552 0.5164473684210527
1520/1520 [==============================] - 0s 23us/step
0.9929133051320126 0.5190789473684211
1520/1520 [==============================] - 0s 27us/step
0.9958901141819201 0.5138157894736842
1520/1520 [==============================] - 0s 27us/step
0.9892274078569914 0.5236842105263158
1520/1520 [==============================] - 0s 27us/step
1.0062086996279265 0.5078947368421053
Final accuracy: 0.515921052631579


In [11]:
val_loss, val_acc = model.evaluate(X_test_norm, y_test)
print(val_loss, val_acc)

1520/1520 [==============================] - 0s 32us/step
1.0062086996279265 0.5078947368421053


In [12]:
predictions = model.predict_classes(X_test_norm)
preds = list(predictions)
pd.Series(list(predictions)).value_counts()

0    679
2    569
1    272
dtype: int64

In [13]:
predictions[0]

1

In [14]:
correct_pred_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred == real:
        correct_pred_dict[pred] += 1
correct_pred_dict

{0: 310, 1: 81, 2: 381}

In [15]:
outcomes_incorrectly_caught_dict = {0:0,1:0,2:0}
for i in range(len(preds)):
    pred = preds[i]
    real = y_test[i]
    if pred != real:
        outcomes_incorrectly_caught_dict[real] += 1
outcomes_incorrectly_caught_dict

{0: 150, 1: 291, 2: 307}